# Draft

In the following notebook I finally get around to trying out
the [Frames library](https://github.com/acowley/Frames), which is
an attempt to provide data frame-like capabilities (from 
[R](http://www.r-tutor.com/r-introduction/data-frame)
and 
[pandas/Python](http://pandas.pydata.org/))
in Haskell. This library is quite new, and at the time
of writing, is not available on Hackage; a direct
installation via GitHub is required:

```
% git clone https://github.com/acowley/Frames.git
% cabal sandbox add-source Frames
% cabal install Frames foldl lens-family --dry-run
% cabal install Frames foldl lens-family
```

The following is stronly based on the
[Frames tutorial](http://acowley.github.io/Frames/), but using
the ARF from 
[the previous notebook](http://htmlpreview.github.io/?https://raw.githubusercontent.com/DougBurke/astro-haskell/master/html/a%20FITSfull%20of%20ARF.html)
as the data source. As I haven't *quite* got around to writing
a full FITS parser for Haskell just yet, I am going to "cheat"
and convert the ARF to a CSV file using tools from 
[CIAO](http://cxc.harvard.edu/ciao/):

 - copy the FITS file (`src.arf`) to an ASCII "almost-CSV" file (`arf.csv`)
 
```
% dmcopy src.arf "arf.csv[opt kernel=text,sep=',']"
% head -5 arf.csv 
#TEXT/SIMPLE
#,ENERG_LO,ENERG_HI,SPECRESP
0.3000000, 0.3100000, 4.874343
0.3100000, 0.3200000, 14.82926
0.3200000, 0.3300000, 21.30229
```

 - manually edit the file to remove the first line and fix up the column names

```
% head -5 arf.csv 
ENERG_LO,ENERG_HI,SPECRESP
0.3000000, 0.3100000, 4.874343
0.3100000, 0.3200000, 14.82926
0.3200000, 0.3300000, 21.30229
0.3300000, 0.3400000, 28.51495
```

For the purposes of this notebook the file is accessed as `../data/arf.csv`.

Using the Frames library requires a selection of
[GHC extensions](https://downloads.haskell.org/~ghc/7.8-latest/docs/html/users_guide/ghc-language-features.html).
When used in source code the form

```
{-# LANGUAGE extension1, ..., extensionN #-}
```

is used, but for the IHaskell notebook it's a bunch of `:set -Xextension1` calls. It's not obvious if I need
all these in this notebook.

In [1]:
-- taken from the tutorial: http://acowley.github.io/Frames/

{-# LANGUAGE ConstraintKinds, DataKinds, FlexibleContexts, GADTs,
             OverloadedStrings, PatternSynonyms, QuasiQuotes,
             ScopedTypeVariables, TemplateHaskell, TypeOperators,
             ViewPatterns #-}

:set -XConstraintKinds
:set -XDataKinds
-- :set -XFlexibleContexts
:set -XGADTs
-- :set -XOverloadedStrings
-- :set -XPatternSynonyms
-- :set -XQuasiQuotes
-- :set -XScopedTypeVariables
-- :set -XTemplateHaskell
-- :set -XTypeOperators
-- :set -XViewPatterns

Here are some of the modules that I will use later on:

In [2]:
import qualified Control.Foldl as L
import qualified Data.Foldable as F

import Control.Applicative

import Data.Proxy (Proxy(..))

import Lens.Family

import Frames
import Frames.CSV (readTableOpt)

The "trick" of the Frames package - i.e. the way that it provides a typed data frame - is that
data loading is separated into two parts:

  - determination of the column names and column types (e.g. the "schema" for the data)
  - reading in the data using this "schema" (this can be done in a "streaming" manner,
    i.e. chunks of rows at a time to support handling very-large files, or all the
    data can be read in one go)

So, the "evaluation" of the CSV file is handled by the 
`tableTypes` function, which is given the name of the type which will
be used to represent a row, and the path to the CSV file (there's
a variant where more control is provided to the user, but I'm using
the "easy" version here).

In [3]:
tableTypes "ARF" "../data/arf.csv"

Once this has succeeded, the compiler will create an `ARF` type that
represents the data in the file. This is handled by
[Template Haskell](https://wiki.haskell.org/Template_Haskell),
which is a GHC extension that provides compile-time metaprogramming
to Haskell. Since I am using `IHaskell`, the fact that this is done
compile-time versus evaluation-time is somewhat hidden from me
(until, if you are anything like me, you accidentally give `tableTypes` an invalid file path
and then wonder why the Haskell kernel dies on you!).

The first argument has to be capitalized, since it represents the
name of a Haskell type (which are syntactically restricted to
starting with a capital letter). Let's look at what the compiler
has worked out for the `ARF` type:

In [4]:
:opt no-pager

In [5]:
:info ARF

type ARF = Record '["ENERG_LO" :-> Double, "ENERG_HI" :-> Double, "SPECRESP" :-> Double] -- Defined at :1:1

So, a row is being modelled by a `Record` with what appears to be a list$^\dagger$ of three
`name :-> type` values, which is how "named" fields are represented in `Frames`.
This is a different approach to the 
[standard Haskell record type](http://learnyouahaskell.com/making-our-own-types-and-typeclasses#record-syntax)
which I used in the previous notebook
to represent XXX-sometihng-or-other-very-interesting-XXX.

Some form of [Heterogenous lists](https://wiki.haskell.org/Heterogenous_collections)

XXX

$^\dagger$ This is not a "normal" list since it starts with "`'[`" rather than just "`[`". The
single quote character makes all the difference, and indicates that this is a
type-level list. If you look closely, you may just spot a few more single
quote characters popping up in this notebook.

The names are the column names, and their types are all
`Double`, which suggests that it's recognized that eash row has three elements:
`ENERG_LO`, `ENERG_HI`, and `SPECRESP`, each of type `Double`.

XXX comment on "list-like" and Record

XXX got to here

The `ARF` type is not the only thing that the call to `tableTypes` 
has created. One of the other items is the set of options needed
to parse the CSV file. In this case, the generated name is 
`aRFParser` (since it's a function or value it has to start
with a lower-case character; in this particular case the resulting
name is perhaps not-that elegant).

In [6]:
:t aRFParser

aRFParser :: ParserOptions

This parser is used with `readTableOpt` to read in the data. As the variable name
(`arfStream`) suggests, this supports a streaming approach
(i.e. is useful for handling large data sets, that you may not want to read
in all in one go),
built on the 
[Pipes](https://hackage.haskell.org/package/pipes-4.1.5/docs/Pipes-Tutorial.html)
library.

XXX 

In [7]:
arfStream = readTableOpt aRFParser "../data/arf.csv"

The type is intimidating...

In [8]:
:type arfStream

arfStream :: forall (m :: * -> *) (rs :: [*]). (ReadRec rs, MonadIO m) => Producer (Record rs) m ()

but fortunately we can ignore this and just 
convert it into an "in-memory" representation (in this case, an "Array of Structures")
using the `inCoreAoS` function.

In [9]:
-- signature needed for the type inference
arf <- inCoreAoS arfStream :: IO (Frame ARF)

In [10]:
:type arf

arf :: Frame ARF

XXX What can I do with this?

I can get back the column headers, in a round about way (for those people used to using 
Python or R):

In [64]:
columnHeaders (Proxy :: Proxy ARF)

["ENERG_LO","ENERG_HI","SPECRESP"]

This has actually worked out the column headers from the *type*, and not from
the value (since `arf` isn't passed to `columnHeaders`). 

XXX not sure where I'm going with this or if it's worth it

In [63]:
:info Record

type Record = Data.Vinyl.Core.Rec Data.Vinyl.Functor.Identity -- Defined in ‘Frames.Rec’

In [31]:
:type columnHeaders

columnHeaders :: forall (cs :: [*]) (proxy :: * -> *) (f :: * -> *). ColumnHeaders cs => proxy (Rec f cs) -> [String]

Almost certainly better to show data access and then perhaps some of the types
than the current approach!

In [11]:
:type view

view :: forall b a a' b'. FoldLike b a a' b b' -> a -> b

Argh: my eyes - `sPECRESP` is not a visually-engaging symbol...

In [12]:
:type view sPECRESP

view sPECRESP :: forall (rs0 :: [*]). RElem SPECRESP rs0 (RIndex SPECRESP rs0) => Record rs0 -> Double

In [13]:
:type view sPECRESP <$> arf

view sPECRESP <$> arf :: Frame Double

In [14]:
view sPECRESP <$> arf

No instance for (Show (Frame Double)) arising from a use of ‘print’
In a stmt of an interactive GHCi command: print it

So, this is "projecting" out the `SPECRESP` field from the row, creating a frame with only this content:

In [62]:
mapM_ print (take 3 (F.toList (view sPECRESP <$> arf)))

4.874343
14.82926
21.30229

How about summary statistics? This can often achieved with a `fold`
over the structure:

In [15]:
:type L.fold
:type L.fold L.minimum
:type L.fold L.minimum (view sPECRESP <$> arf)

L.fold :: forall a b (f :: * -> *). Foldable f => Fold a b -> f a -> b

L.fold L.minimum :: forall a (f :: * -> *). (Foldable f, Ord a) => f a -> Maybe a

L.fold L.minimum (view sPECRESP <$> arf) :: Maybe Double

In [16]:
L.fold L.minimum (view sPECRESP <$> arf)
L.fold L.maximum (view sPECRESP <$> arf)

Just 0.5657126

Just 672.1996

Note that the following does *not* evaluate the structure twice (once for
`minimum` and once for `maximum`):

In [17]:
-- from the tutorial
minMax :: Ord a => L.Fold a (Maybe a, Maybe a)
minMax = (,) <$> L.minimum <*> L.maximum

In [18]:
L.fold minMax (view sPECRESP <$> arf)

(Just 0.5657126,Just 672.1996)

In [19]:
:type L.pretraverse sPECRESP

L.pretraverse sPECRESP :: forall r (rs0 :: [*]). RElem SPECRESP rs0 (RIndex SPECRESP rs0) => Fold Double r -> Fold (Record rs0) r

In [20]:
:type L.pretraverse sPECRESP minMax 

L.pretraverse sPECRESP minMax :: forall (rs0 :: [*]). RElem SPECRESP rs0 (RIndex SPECRESP rs0) => Fold (Record rs0) (Maybe Double, Maybe Double)

In [21]:
L.fold (L.pretraverse sPECRESP minMax) arf

(Just 0.5657126,Just 672.1996)

Note that in the following the columns include their names (or, rather, the fields
in each record). This can be compared to earlier when `view sPECRESP` was used to
"extract" out the contents of the `SPECRESP` field, but losing the name):

In [22]:
mapM_ print (take 3 (F.toList arf))

{ENERG_LO :-> 0.3, ENERG_HI :-> 0.31, SPECRESP :-> 4.874343}
{ENERG_LO :-> 0.31, ENERG_HI :-> 0.32, SPECRESP :-> 14.82926}
{ENERG_LO :-> 0.32, ENERG_HI :-> 0.33, SPECRESP :-> 21.30229}

In [65]:
mapM_ print (take 3 (F.toList (view sPECRESP <$> arf)))

4.874343
14.82926
21.30229

XXX can I just extract `SPECRESP` record (i.e. include 'SPECRESP :-> xxx') XXX see tutorial XXX

In [23]:
nrows = frameLength arf
mapM_ (print . frameRow arf) [nrows - 3 .. nrows - 1]

{ENERG_LO :-> 10.97, ENERG_HI :-> 10.98, SPECRESP :-> 0.6049552}
{ENERG_LO :-> 10.98, ENERG_HI :-> 10.99, SPECRESP :-> 0.5852773}
{ENERG_LO :-> 10.99, ENERG_HI :-> 11.0, SPECRESP :-> 0.5657126}

In [24]:
-- subsets
mapM_ print (take 5 (filter ((>= 600) . view sPECRESP) (F.toList arf)))

{ENERG_LO :-> 1.18, ENERG_HI :-> 1.19, SPECRESP :-> 601.2171}
{ENERG_LO :-> 1.19, ENERG_HI :-> 1.2, SPECRESP :-> 604.2618}
{ENERG_LO :-> 1.2, ENERG_HI :-> 1.21, SPECRESP :-> 607.2027}
{ENERG_LO :-> 1.21, ENERG_HI :-> 1.22, SPECRESP :-> 610.0424}
{ENERG_LO :-> 1.22, ENERG_HI :-> 1.23, SPECRESP :-> 612.9407}

In [25]:
-- would be nice to have a HTML representation of the table;
-- how to get column names (see above)?
arf

No instance for (Show (Frame ARF)) arising from a use of ‘print’
In a stmt of an interactive GHCi command: print it

In [26]:
-- and how to extract the data to plot it? Frames has plot examples

In [27]:
:t recUncons

recUncons :: forall (s :: Symbol) a (rs :: [*]). Record ((s :-> a) : rs) -> (a, Record rs)

In [28]:
-- this gives the value, but not the name
(a1,b1) = recUncons (frameRow arf 0)
(a2,b2) = recUncons b1
(a3,b3) = recUncons b2

(a1,a2,a3)

(0.3,0.31,4.874343)

In [29]:
b1
b2
b3

{ENERG_HI :-> 0.31, SPECRESP :-> 4.874343}

{SPECRESP :-> 4.874343}

{}

In [30]:
-- as expected, this falls over
recUncons b3

Couldn't match type ‘'[]’ with ‘(s0 :-> a) : rs’
Expected type: Record ((s0 :-> a) : rs)
  Actual type: Record '[]
Relevant bindings include it :: (a, Record rs) (bound at <interactive>:1:1)
In the first argument of ‘recUncons’, namely ‘b3’
In the expression: recUncons b3

In [34]:
showFields (frameRow arf 0)

["0.3","0.31","4.874343"]

In [35]:
toVinyl (frameRow arf 0)

{0.3, 0.31, 4.874343}

In [36]:
:type toVinyl (frameRow arf 0)

toVinyl (frameRow arf 0) :: Rec Identity (UnColumn '["ENERG_LO" :-> Double, "ENERG_HI" :-> Double, "SPECRESP" :-> Double])

In [37]:
:type frameRow

frameRow :: forall r. Frame r -> Int -> r

In [38]:
:info ARF

type ARF = Record '["ENERG_LO" :-> Double, "ENERG_HI" :-> Double, "SPECRESP" :-> Double] -- Defined at :1:1

Let's leave making a nice HTML table for now.

From `Frames.Col`:

```
instance forall s a. (KnownSymbol s, Show a) => Show (s :-> a) where
  show (Col x) = symbolVal (Proxy::Proxy s)++" :-> "++show x
```

In [39]:
:info ENERGLO

type ENERGLO = "ENERG_LO" :-> Double -- Defined at :1:1

In [40]:
type ENERGY = "ENERGY" :-> Double

In [41]:
rnew :: Record [ENERGY, ENERGLO, ENERGHI, SPECRESP]
rnew = frameCons (pure 0.305) (frameRow arf 0)

In [42]:
rnew

{ENERGY :-> 0.305, ENERG_LO :-> 0.3, ENERG_HI :-> 0.31, SPECRESP :-> 4.874343}

In [43]:
:info ARF

type ARF = Record '["ENERG_LO" :-> Double, "ENERG_HI" :-> Double, "SPECRESP" :-> Double] -- Defined at :1:1

In [44]:
:type rnew

rnew :: Record '[ENERGY, ENERGLO, ENERGHI, SPECRESP]

In [45]:
-- would it be easy to add in the column after ENERGHI, say?
type ARFmod = Record [ENERGY, ENERGLO, ENERGHI, SPECRESP]

In [46]:
-- Ideally would only require the lo/hi energy fields;
-- => addEnergyCol2
addEnergyCol :: Record [ENERGLO, ENERGHI, SPECRESP] -> ARFmod
addEnergyCol r = 
  let elo = view eNERGLO r
      ehi = view eNERGHI r
      emid = (elo + ehi) / 2
  in frameCons (pure emid) r

In [47]:
addEnergyCol2 :: 
  (ENERGLO ∈ rs, ENERGHI ∈ rs) 
  => Record rs
  -> Record (ENERGY ': rs)
addEnergyCol2 r = 
  let elo = view eNERGLO r
      ehi = view eNERGHI r
      emid = (elo + ehi) / 2
  in frameCons (pure emid) r

In [48]:
:type frameCons

frameCons :: forall (f :: * -> *) a (rs :: [*]) (s :: Symbol). Functor f => f a -> Rec f rs -> Rec f ((s :-> a) : rs)

In [68]:
{-

Not sure this is a worthwhile avenue

-- try and avoid types where necessary; I was hoping just to be able to give
-- a type to the new column, but it isn't obvious if it's possible...
addEnergyCol3 r = 
  let elo = view eNERGLO r
      ehi = view eNERGHI r
      emid = (elo + ehi) / 2
      newcol :: ENERGY ????XXXXX what goes here
      newcol = pure emid
  in frameCons newcol r
  
  
-}  

In [50]:
addEnergy :: Frame ARF -> Frame ARFmod
addEnergy = fmap addEnergyCol

In [51]:
import qualified Pipes.Prelude as P
import Pipes hiding (Proxy)

In [52]:
{-

writers :: (Occupation ∈ rs, Monad m) => Pipe (Record rs) (Record rs) m r
writers = P.filter ((== "writer") . view occupation)

-}

-- for now be very specific with the types
addEnergyP :: Monad m => Pipe ARF ARFmod m r
addEnergyP = P.map addEnergyCol

In [53]:
runEffect (arfStream >-> addEnergyP >-> P.take 6 >-> P.print)

{ENERGY :-> 0.305, ENERG_LO :-> 0.3, ENERG_HI :-> 0.31, SPECRESP :-> 4.874343}
{ENERGY :-> 0.315, ENERG_LO :-> 0.31, ENERG_HI :-> 0.32, SPECRESP :-> 14.82926}
{ENERGY :-> 0.325, ENERG_LO :-> 0.32, ENERG_HI :-> 0.33, SPECRESP :-> 21.30229}
{ENERGY :-> 0.335, ENERG_LO :-> 0.33, ENERG_HI :-> 0.34, SPECRESP :-> 28.51495}
{ENERGY :-> 0.345, ENERG_LO :-> 0.34, ENERG_HI :-> 0.35, SPECRESP :-> 35.39883}
{ENERGY :-> 0.355, ENERG_LO :-> 0.35, ENERG_HI :-> 0.36, SPECRESP :-> 41.54232}

In [54]:
arfMod = addEnergy arf

In [55]:
mapM_ print (take 6 (F.toList arfMod))

{ENERGY :-> 0.305, ENERG_LO :-> 0.3, ENERG_HI :-> 0.31, SPECRESP :-> 4.874343}
{ENERGY :-> 0.315, ENERG_LO :-> 0.31, ENERG_HI :-> 0.32, SPECRESP :-> 14.82926}
{ENERGY :-> 0.325, ENERG_LO :-> 0.32, ENERG_HI :-> 0.33, SPECRESP :-> 21.30229}
{ENERGY :-> 0.335, ENERG_LO :-> 0.33, ENERG_HI :-> 0.34, SPECRESP :-> 28.51495}
{ENERGY :-> 0.345, ENERG_LO :-> 0.34, ENERG_HI :-> 0.35, SPECRESP :-> 35.39883}
{ENERGY :-> 0.355, ENERG_LO :-> 0.35, ENERG_HI :-> 0.36, SPECRESP :-> 41.54232}

Can I write a model function that requires an ARF-like record and caluclutes a model?

In [56]:
-- Note, since the "columns" are typed, in that SPECRESP/...
-- are Doubles, this is not polymorphic.
--
-- This is an "integrated" model
powerLaw1 ::   
  (ENERGLO ∈ rs, ENERGHI ∈ rs, SPECRESP ∈ rs) 
  => Double      -- ^ amplitude at 1 keV
  -> Double      -- ^ gamma
  -> Record rs
  -> Double      -- ^ units of the amplitude * Energy
powerLaw1 norm gamma rs = 
  let elo = view eNERGLO rs
      ehi = view eNERGHI rs
      p   = 1 - gamma
      val = if gamma == 1
            then log (ehi/elo)
            else (ehi**p - elo**p) / p
             
  in norm * val

Perhaps I want to add the model value to the record rather than just return a `Double`;
in this case I think the interface would be that there's a function to do this,
and it accepts a generic model (with an interface more like the ones shown
before, i.e. something that doesn't know about records).

In [57]:
type IModelFunc = Double -> Double -> Double

type IModel = "IModel" :-> Double

In [58]:
evalModel :: 
  (ENERGLO ∈ rs, ENERGHI ∈ rs, SPECRESP ∈ rs) 
  => IModelFunc
  -> Record rs
  -> Double
evalModel f r = 
  let elo = view eNERGLO r
      ehi = view eNERGHI r
  in f elo ehi
  
addModel :: 
  (ENERGLO ∈ rs, ENERGHI ∈ rs, SPECRESP ∈ rs) 
  => IModelFunc
  -> Record rs
  -> Record (IModel ': rs)
addModel f r = frameCons (pure (evalModel f r)) r

In [59]:
powerLaw :: 
  Double         -- ^ normalization 
  -> Double      -- ^ gamma 
  -> IModelFunc
powerLaw norm gamma elo ehi | gamma == 1 = norm * log (ehi / elo)
                            | otherwise  = let p = 1 - gamma
                                           in norm * (ehi**p - elo**p) / p

In [60]:
model = fmap (addModel (powerLaw 1.0 2.0)) arf

In [61]:
mapM_ print (take 6 (F.toList model))

{IModel :-> 0.10752688172043001, ENERG_LO :-> 0.3, ENERG_HI :-> 0.31, SPECRESP :-> 4.874343}
{IModel :-> 0.10080645161290347, ENERG_LO :-> 0.31, ENERG_HI :-> 0.32, SPECRESP :-> 14.82926}
{IModel :-> 9.469696969696972e-2, ENERG_LO :-> 0.32, ENERG_HI :-> 0.33, SPECRESP :-> 21.30229}
{IModel :-> 8.912655971479522e-2, ENERG_LO :-> 0.33, ENERG_HI :-> 0.34, SPECRESP :-> 28.51495}
{IModel :-> 8.403361344537785e-2, ENERG_LO :-> 0.34, ENERG_HI :-> 0.35, SPECRESP :-> 35.39883}
{IModel :-> 7.936507936507953e-2, ENERG_LO :-> 0.35, ENERG_HI :-> 0.36, SPECRESP :-> 41.54232}

Back to an earlier note about wanting to easily extract a subset of columns; need to look
at the tutorial.